In [15]:
#!/usr/bin/env python3

from dotenv import load_dotenv
import os
# from pprint import pprint
from tikapi import TikAPI, ValidationException, ResponseException
from rich.pretty import pprint
from rich.console import Console
console = Console()
import requests

load_dotenv('.env.local')
token = os.environ.get("tik_api_key")
api = TikAPI(token)

def p(object):
    pprint(object, max_depth=2, console=console)

def downloadfile(name,url):
    name=name+".mp4"
    r=requests.get(url)
    print("****Connected****")
    f=open(name,'wb')
    print("Donloading.....")
    for chunk in r.iter_content(chunk_size=255): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
            # print(chunk)
    print("Done")
    f.close()


try:
    response = api.public.hashtag(
        name="gym"
    )

    hashtagId = response.json()['challengeInfo']['challenge']['id']

    response = api.public.hashtag(
        id=hashtagId
    )


    # pprint(response.json())
    items = response.json()['itemList']
    # for item in items:
    #     p('NEW ITEM')
    #     p(item)

    item = items[6]
    pprint(item)
    p(item['textExtra'])
    downloadAddr = item['video']['downloadAddr']
    name = item['video']['id']
    downloadfile(name, downloadAddr)
    response.save_video(downloadAddr, name+'.mp4')
    

    # print(response)



# iteration
    # while(response):
    #     cursor = response.json().get('cursor')
    #     print("Getting next items ", cursor)
    #     response = response.next_items()

except ValidationException as e:
    print(e, e.field)

except ResponseException as e:
    print(e, e.response.status_code)

{
│   'BAInfo': '',
│   'adAuthorization': False,
│   'adLabelVersion': 0,
│   'author': {
│   │   'avatarLarger': 'https://p16-sign-va.tiktokcdn.com/musically-maliva-obj/1622908554629125~c5_1080x1080.jpeg?x-expires=1673830800&x-signature=ossvZ250deJ%2Fpj7Lt5POgOE0P9g%3D',
│   │   'avatarMedium': 'https://p16-sign-va.tiktokcdn.com/musically-maliva-obj/1622908554629125~c5_720x720.jpeg?x-expires=1673830800&x-signature=acYy3XaYAKCr9OtmW%2BXNcXEhCIw%3D',
│   │   'avatarThumb': 'https://p16-sign-va.tiktokcdn.com/musically-maliva-obj/1622908554629125~c5_100x100.jpeg?x-expires=1673830800&x-signature=Sg3GiLTAMibbsUttoURKDzNQfHI%3D',
│   │   'commentSetting': 0,
│   │   'downloadSetting': 0,
│   │   'duetSetting': 0,
│   │   'ftc': False,
│   │   'id': '18702747',
│   │   'isADVirtual': False,
│   │   'isEmbedBanned': False,
│   │   'nickname': 'patroX',
│   │   'openFavorite': False,
│   │   'privateAccount': False,
│   │   'relation': 0,
│   │   'secUid': 'MS4wLjABAAAAiFnldaILebi5heDoVU6bn4jBWWycX6-9U3xuNPqZ8Ws',
│   │   'secret': False,
│   │   'signature': '💡 CEO of Chestday & Cringe\n🔥 Masters of Dance winner (Tv Show)\n❤️ Happiness',
│   │   'stitchSetting': 0,
│   │   'ttSeller': False,
│   │   'uniqueId': 'patrox',
│   │   'verified': True
│   },
│   'authorStats': {
│   │   'diggCount': 2313,
│   │   'followerCount': 8400000,
│   │   'followingCount': 714,
│   │   'heart': 146100000,
│   │   'heartCount': 146100000,
│   │   'videoCount': 1152
│   },
│   'challenges': [
│   │   {
│   │   │   'coverLarger': '',
│   │   │   'coverMedium': '',
│   │   │   'coverThumb': '',
│   │   │   'desc': '',
│   │   │   'id': '35117',
│   │   │   'isCommerce': False,
│   │   │   'profileLarger': '',
│   │   │   'profileMedium': '',
│   │   │   'profileThumb': '',
│   │   │   'stats': {'videoCount': 0, 'viewCount': 0},
│   │   │   'title': 'chestday'
│   │   },
│   │   {
│   │   │   'coverLarger': '',
│   │   │   'coverMedium': '',
│   │   │   'coverThumb': '',
│   │   │   'desc': '',
│   │   │   'id': '42802800',
│   │   │   'isCommerce': False,
│   │   │   'profileLarger': '',
│   │   │   'profileMedium': '',
│   │   │   'profileThumb': '',
│   │   │   'stats': {'videoCount': 0, 'viewCount': 0},
│   │   │   'title': 'patrox'
│   │   },
│   │   {
│   │   │   'coverLarger': '',
│   │   │   'coverMedium': '',
│   │   │   'coverThumb': '',
│   │   │   'desc': '',
│   │   │   'id': '229207',
│   │   │   'isCommerce': False,
│   │   │   'profileLarger': '',
│   │   │   'profileMedium': '',
│   │   │   'profileThumb': '',
│   │   │   'stats': {'videoCount': 0, 'viewCount': 0},
│   │   │   'title': 'fyp'
│   │   },
│   │   {
│   │   │   'coverLarger': '',
│   │   │   'coverMedium': '',
│   │   │   'coverThumb': '',
│   │   │   'desc': '',
│   │   │   'id': '153828',
│   │   │   'isCommerce': False,
│   │   │   'profileLarger': '',
│   │   │   'profileMedium': '',
│   │   │   'profileThumb': '',
│   │   │   'stats': {'videoCount': 0, 'viewCount': 0},
│   │   │   'title': 'fy'
│   │   },
│   │   {
│   │   │   'coverLarger': '',
│   │   │   'coverMedium': '',
│   │   │   'coverThumb': '',
│   │   │   'desc': "It's #Fail time",
│   │   │   'id': '5013',
│   │   │   'isCommerce': False,
│   │   │   'profileLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva-obj/3a21ea416880cd7c9f6007a8efc8283a',
│   │   │   'profileMedium': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva-obj/3a21ea416880cd7c9f6007a8efc8283a',
│   │   │   'profileThumb': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva-obj/3a21ea416880cd7c9f6007a8efc8283a',
│   │   │   'stats': {'videoCount': 0, 'viewCount': 0},
│   │   │   'title': 'fail'
│   │   },
│   │   {
│   │   │   'coverLarger': '',
│   │   │   'coverMedium': '',
│   │   │   'coverThumb': '',
│   │   │   'desc': 'From exercises to gym motivation and memes, show us how you workout!',
│   │   │   'id': '7882',
│   │   │   'isCommerce': False,
│   │   │   'profileLarger': 'https://p16-amd-va.tiktokcdn.com/obj

[
│   {
│   │   'awemeId': '',
│   │   'end': 69,
│   │   'hashtagId': '',
│   │   'hashtagName': '',
│   │   'isCommerce': False,
│   │   'secUid': 'MS4wLjABAAAAI_-a9_RDeWt0yiRzoqLbP57MJzQDqt18inT1q_-uyyWoE8JyutcT7l390bKMNj6G',
│   │   'start': 51,
│   │   'subType': 0,
│   │   'type': 0,
│   │   'userId': '6809661425432429574',
│   │   'userUniqueId': 'paul_unterleitner'
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 104,
│   │   'hashtagId': '',
│   │   'hashtagName': '',
│   │   'isCommerce': False,
│   │   'secUid': 'MS4wLjABAAAAheepka3tUlErU39dnDUSPF0FlRWUONLdj9M2EHFRcpysz08ZAdNRVeQ9B6Hb0xpQ',
│   │   'start': 90,
│   │   'subType': 0,
│   │   'type': 0,
│   │   'userId': '114594966600925184',
│   │   'userUniqueId': 'johnymuenster'
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 112,
│   │   'hashtagId': '',
│   │   'hashtagName': '',
│   │   'isCommerce': False,
│   │   'secUid': 'MS4wLjABAAAAMOJuMLJ8AK0Y4nOlmL5RujiBkyyiurJk-sZderDJ4KiDjLuvxVjubJ3jI3fE4UKk',
│   │   'start': 105,
│   │   'subType': 0,
│   │   'type': 0,
│   │   'userId': '6744803684313629701',
│   │   'userUniqueId': 'niemka'
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 122,
│   │   'hashtagId': '35117',
│   │   'hashtagName': 'chestday',
│   │   'isCommerce': False,
│   │   'secUid': '',
│   │   'start': 113,
│   │   'subType': 0,
│   │   'type': 1,
│   │   'userId': '',
│   │   'userUniqueId': ''
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 130,
│   │   'hashtagId': '42802800',
│   │   'hashtagName': 'patrox',
│   │   'isCommerce': False,
│   │   'secUid': '',
│   │   'start': 123,
│   │   'subType': 0,
│   │   'type': 1,
│   │   'userId': '',
│   │   'userUniqueId': ''
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 135,
│   │   'hashtagId': '229207',
│   │   'hashtagName': 'fyp',
│   │   'isCommerce': False,
│   │   'secUid': '',
│   │   'start': 131,
│   │   'subType': 0,
│   │   'type': 1,
│   │   'userId': '',
│   │   'userUniqueId': ''
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 139,
│   │   'hashtagId': '153828',
│   │   'hashtagName': 'fy',
│   │   'isCommerce': False,
│   │   'secUid': '',
│   │   'start': 136,
│   │   'subType': 0,
│   │   'type': 1,
│   │   'userId': '',
│   │   'userUniqueId': ''
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 145,
│   │   'hashtagId': '5013',
│   │   'hashtagName': 'fail',
│   │   'isCommerce': False,
│   │   'secUid': '',
│   │   'start': 140,
│   │   'subType': 0,
│   │   'type': 1,
│   │   'userId': '',
│   │   'userUniqueId': ''
│   },
│   {
│   │   'awemeId': '',
│   │   'end': 150,
│   │   'hashtagId': '7882',
│   │   'hashtagName': 'gym',
│   │   'isCommerce': False,
│   │   'secUid': '',
│   │   'start': 146,
│   │   'subType': 0,
│   │   'type': 1,
│   │   'userId': '',
│   │   'userUniqueId': ''
│   }
]

****Connected****
Donloading.....
Done


In [10]:
import ffmpeg

(
    ffmpeg
    .input(name+'.mp4')
    .output(name+'.mp3')
    .run()
)


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack 

(None, None)

In [12]:
import whisper
from whisper.utils import exact_div, format_timestamp, optional_int, optional_float, str2bool, write_txt, write_vtt, write_srt


model = whisper.load_model("small")
result = model.transcribe(name+".mp3")
# print(result["text"])
# f= open(name+'.txt',"w+")
# f.write(result['text'])
# f.close()
with open(os.path.join(name + ".txt"), "w", encoding="utf-8") as txt:
            write_txt(result["segments"], file=txt)


